In [1]:
import base64
import pandas as pd
import requests
import json
import spotipy.util as util
import spotipy

In [2]:
# import secrets for toke authenification 
import import_ipynb
from spotify_secret import *

importing Jupyter notebook from spotify_secret.ipynb


In [3]:
# required information for a authentication token 
scopes = ['user-read-recently-played', 'playlist-modify-private']

In [291]:
# get access toke (tokens only last 1 hour)
token = util.prompt_for_user_token(username=username, 
                                   scope=scopes, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth = token)

### Recently played

In [293]:
# url for recently played
spotify_url = 'https://api.spotify.com/v1/me/player/recently-played'
#header for all api interactions
headers = {
    'Authorization': 'Bearer ' + token,
    'Content-Type': 'application/json',
}

In [6]:
# recent tracks
request = requests.get(url = spotify_url, headers=headers)
recent = request.json()

### My Data 

In [268]:
# my library
with open('YourLibrary.json') as json_data:
    data = json.load(json_data)
my_library = pd.DataFrame(data['tracks'])
my_library['art_track'] = my_library['artist'] + my_library['track']

In [269]:
# my listener history as a df 
files = ['StreamingHistory0.json', 'StreamingHistory1.json']
list_of_df = []
for file in files:
    with open(file, encoding="utf8") as stream_json:
        stream_data = json.load(stream_json)
        my_streams = pd.DataFrame(stream_data)
        list_of_df.append(my_streams)
full_streams = pd.concat(list_of_df)

In [270]:
my_full_streams = full_streams

In [271]:
my_full_streams['msPlayed'] = (my_full_streams['msPlayed'] / 1000)
my_full_streams = my_full_streams.rename(columns={'msPlayed' : 'Secs_played'})
my_full_streams['art_track'] = my_full_streams['artistName'] + my_full_streams['trackName']
my_full_streams.head(5)

,endTime,artistName,trackName,Secs_played,art_track
0,2022-06-11 23:03,Musical Youth,Pass The Dutchie,185.152,Musical YouthPass The Dutchie
1,2022-06-12 13:21,Doomtree,Slow Burn,236.592,DoomtreeSlow Burn
2,2022-06-12 13:24,Grieves,What It Dew,214.893,GrievesWhat It Dew
3,2022-06-12 13:28,Factor Chandelier,Feeling Inside,236.787,Factor ChandelierFeeling Inside
4,2022-06-12 13:31,GDP,Butter Scripts,185.560,GDPButter Scripts


In [272]:
my_full_streams.shape

(13695, 5)

In [273]:
no_skips = my_full_streams[my_full_streams['Secs_played'] > 15]
no_skips.shape

(7975, 5)

In [274]:
freq_df = no_skips[['trackName','artistName', 'art_track']]

In [275]:
freq_df['count'] = freq_df.groupby('art_track')['art_track'].transform('count')

C:\Users\willd\AppData\Local\Temp\ipykernel_10136\454162367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  freq_df['count'] = freq_df.groupby('art_track')['art_track'].transform('count')


In [276]:
freq_df.shape

(7975, 4)

In [277]:
freq_df = freq_df.drop_duplicates()
freq_df.shape

(3206, 4)

In [278]:
over_five = freq_df[freq_df['count'] >= 5]
over_five.shape

(357, 4)

In [279]:
not_liked = over_five[~over_five['art_track'].isin(my_library['art_track'])]

In [280]:
not_liked = not_liked.reset_index(drop=True)

In [281]:
not_liked['uri'] = ''
# get_uri(sample1['trackName'], sample1['artistName'])

In [284]:
for i in range(0, len(not_liked)):
    not_liked.loc[i,'uri'] = get_uri(not_liked.loc[i,'trackName'], not_liked.loc[i,'artistName'])

In [286]:
liked_not_liked = list(not_liked['uri'])

In [287]:
liked_not_liked

['spotify:track:5w5vw9cokQ9s8dHQmQqLX4',
 'spotify:track:4iKYe4neoybSn78i1AH7z7',
 'spotify:track:7rYGZmhNJhsmVLjPidM1q8',
 'spotify:track:6ey8aat11RgL4jYvC4lRAy',
 'spotify:track:5CUC1EDq80dxXRIrRW1aWA',
 'spotify:track:05cjQouqyDYG6F2NSNJL7E',
 'spotify:track:7n3WO6ESKS1uCI9fgkGs66',
 'spotify:track:4REjaHRPmVb7btssqChJSy',
 'spotify:track:1EjzcBTVLV7ATtdsQwyV31',
 'spotify:track:0PuRzQcyEUBegg5GIWf9no',
 'spotify:track:0dPjbyTqlDkHhfp2amfcWG',
 'spotify:track:27SjutLnh1PXRYDjmSDs78',
 'spotify:track:0xxpfL0b31St9FGTBMgAVG',
 'spotify:track:5gABUTybqevRxpFA9Bl2AT',
 'spotify:track:3NMDFORx33jILYPnSP4Jcg',
 'spotify:track:0ByKkm4qqcDPtC2cPWo1wE',
 'spotify:track:4VgcZBaJAibMnZDEOBQB7O',
 'spotify:track:2fClGossFdEdhjjeYceaN7',
 'spotify:track:68qdRCldPQ0fPORJGtiyrI',
 'spotify:track:561F1zqRwGPCTMRsLsXVtL',
 'spotify:track:3q0wgJXeUkhq2jeZ9JS7KN',
 'spotify:track:4HA2jo7wkMdN1lmLO6ryzE',
 'spotify:track:1Nig8tnYHCBaBz7FPIR763',
 'spotify:track:46A9awz7xtpDGNeNtaqWMh',
 'spotify:track:

### Feature analysis

In [33]:
# get track features
features = sp.audio_features(uris_list)

In [32]:
# make features df
feature_df = pd.DataFrame.from_records(features)
#join to reduced df
full_df = reduced_df.join(feature_df.set_index('uri'), on='uri')

In [ ]:
# transform listener history


### Get required track information

In [185]:
# function to get uris for df
# def get_uri(track, artist):
#     search_song = sp.search(q='track: ' + track, type='track')
#     for option in search_song['tracks']['items']:
#         if option['artists'][0]['name'] == artist:
#             track_uri = option['uri']
#             break
#         else: 
#             track_uri = ""
#     return track_uri

In [251]:
def get_uri(track, artist):
    qq = f""
    search_song = sp.search(q=f'track: {track} artist: {artist}', type='track')
    track_uri=search_song['tracks']['items'][0]['uri']
    return track_uri

In [205]:
search_song=sp.search(q='track: Bad Dreams artist: Lyrics Born', type='track')

In [196]:
# for option in search_song['tracks']['items']:
#     if option['artists'][0]['name'] == 'Lyrics Born':
#         track_uri = option['uri']
        

### Playlist Creation & Modification 

In [289]:
# url for playlist api
playlist_url = 'https://api.spotify.com/v1/users/willdavis111/playlists'
# boilerplate json for playlist creation
pl_name_data = {'name': '', 'description': 'New playlist description','public': False,}
# function to create playlist and return playlist id for modification
def new_playlist(playlist_name):
    global pl_id
    pl_name_data['name'] = playlist_name
    playlist_created = requests.post(playlist_url, headers=headers, json=pl_name_data)
    pl_obj = playlist_created.json()
    pl_id = pl_obj['id']

In [110]:
# token for authentication
sp = spotipy.Spotify(auth = token)
# function to update created plalists (only accepts lists of uris < 100)
def mod_playlist(pl_id, pl_uris):
    sp.playlist_add_items(pl_id, pl_uris, position=None)

In [294]:
# call function to create playlist
new_playlist('Lost Tracks')

In [295]:
# call function to add to newly created playlist 
mod_playlist(pl_id, liked_not_liked)